In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import gensim
USE_CUDA = torch.cuda.is_available()

from data import load_squad_data,preprop,getBatch,pad_to_batch
from model import CoattentionEncoder, DynamicDecoder

In [2]:
MAX_LEN=400

In [3]:
dataset = load_squad_data('dataset/train-v1.1.json',MAX_LEN)
word2index,train_data = preprop(dataset)

Skipped 800, 86655 question/answer
Successfully Build 114855 vocabs
Preprop Complete!


In [4]:
%%time
#python3 -m gensim.scripts.glove2word2vec --input  glove.840B.300d.txt --output glove.840B.300d.w2vformat.txt
model = gensim.models.KeyedVectors.load_word2vec_format('dataset/glove.840B.300d.w2vformat.txt')

CPU times: user 5min 29s, sys: 3.07 s, total: 5min 32s
Wall time: 5min 25s


In [5]:
# oov=[]
# for k in word2index.keys():
#     if k not in ['<pad>','<unk>','<s>','</s>'] and model.vocab.get(k) is None:
#         oov.append(k)
# for o in oov:
#     word2index.pop(o)
# print(len(oov),len(word2index))

22527 92328


In [5]:
pretrained = []

for i in range(len(word2index)):
    try:
        pretrained.append(model[word2index[i]])
    except:
        pretrained.append(np.random.randn(300))
        
pretrained_vectors = np.vstack(pretrained)

In [7]:
# del oov
# del pretrained
# del model

# word2index,train_data = preprop(dataset,word2index)

Successfully Build 92328 vocabs
Preprop Complete!


In [6]:
EMBED_SIZE=300
HIDDEN_SIZE=200
MAXOUT_POOL=4
MAX_ITER=4
BATCH_SIZE=32
STEP=100
LR=0.001
encoder = CoattentionEncoder(len(word2index),EMBED_SIZE,HIDDEN_SIZE,pretrained_wvectors=pretrained_vectors,is_static=False,use_cuda=USE_CUDA)
decoder = DynamicDecoder(HIDDEN_SIZE,MAXOUT_POOL,max_iter=MAX_ITER,use_cuda=USE_CUDA)

loss_function = nn.CrossEntropyLoss()
enc_optim = optim.Adam(filter(lambda p: p.requires_grad, encoder.parameters()),lr=LR)
dec_optim = optim.Adam(decoder.parameters(),lr=LR)

In [ ]:
for step in range(STEP):
    losses=[]
    for i,batch in enumerate(getBatch(BATCH_SIZE,train_data)):
        documents,questions,starts,ends = pad_to_batch(batch,word2index)

        encoder.zero_grad()
        decoder.zero_grad()
        U = encoder(documents,questions,True)
        _,_,entropies = decoder(U,True)

        s_ents, e_ents = list(zip(*entropies)) # x MAX_ITER
        loss_start,loss_end=0,0
        for m in range(MAX_ITER):
            loss_start+=loss_function(s_ents[m],starts.view(-1))
            loss_end+=loss_function(s_ents[m],ends.view(-1))

        loss = loss_start+loss_end
        losses.append(loss.data[0])
        loss.backward()
        torch.nn.utils.clip_grad_norm(encoder.parameters(), 10) # gradient clipping
        torch.nn.utils.clip_grad_norm(decoder.parameters(), 10) 
        enc_optim.step()
        dec_optim.step()
        
        if i % 100 == 0:
            print("[%d/%d] [%d/%d] loss : %.3f" % (step,STEP,i,len(train_data)//BATCH_SIZE,np.mean(losses)))
            losses=[]

[0/100] [0/2707] loss : 46.285
[0/100] [100/2707] loss : 37.049
[0/100] [200/2707] loss : 34.712
[0/100] [300/2707] loss : 33.723
[0/100] [400/2707] loss : 33.253
[0/100] [500/2707] loss : 33.097
[0/100] [600/2707] loss : 32.402
[0/100] [700/2707] loss : 32.549
[0/100] [800/2707] loss : 31.742
[0/100] [900/2707] loss : 31.806
[0/100] [1000/2707] loss : 31.342
[0/100] [1100/2707] loss : 31.173
[0/100] [1200/2707] loss : 30.844
[0/100] [1300/2707] loss : 30.326
[0/100] [1400/2707] loss : 30.153
[0/100] [1500/2707] loss : 29.546
[0/100] [1600/2707] loss : 29.212
[0/100] [1700/2707] loss : 28.608
[0/100] [1800/2707] loss : 28.547
[0/100] [1900/2707] loss : 27.499
[0/100] [2000/2707] loss : 27.465
[0/100] [2100/2707] loss : 26.664
[0/100] [2200/2707] loss : 26.101
[0/100] [2300/2707] loss : 24.922
[0/100] [2400/2707] loss : 24.971
[0/100] [2500/2707] loss : 23.863
[0/100] [2600/2707] loss : 23.626
[0/100] [2700/2707] loss : 23.464
[1/100] [0/2707] loss : 24.473
[1/100] [100/2707] loss : 22.